In [46]:
import pandas as pd
import pickle

from classes.model_factory import ModelFactory

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


## Treinamento classificação

Para a predição de falhas com 20 ciclos de antecedência, temos o seguinte cenário:
* <b>7.457</b> exemplos de máquinas que não irão falhar nos próxios 20 ciclos
* <b>796</b> exemplos que irão falhar
* <b>0</b> (negativo): label que representa a <b>não falha</b> nos próximos 20 ciclos
* <b>1</b> (positivo): label que representa <b>uma falha</b> nos próximos 20 ciclos
* Verdadeiro positivo <b>(VP)</b>: Modelo previu que a máquina iria falhar e acertou
* Falso positivo <b>(FP)</b>: Modelo não previu que máquina iria falhar porém falhou
* Verdadeiro negativo <b>(VN)</b>: Modelo previu que a máquina não iria falhar e não falhou
* Falso negativo <b>(FN)</b>: Modelo previu que a máquina iria falhar porém ela não falhou

#### - Nesse contexto de negócio os <b>FPs</b> (não previu a falha) são os mais críticos, pois implica no custo de parar a linha de produção.
#### - Os <b>FNs</b> (troca desnecessária de máquina) são os segundo mais críticos, pois implica o custo de substituir uma máquina com o tempo de uso maior que 20 ciclos

#### - Dito isso, esses são os principais pontos que irei observar para selecionar o melhor modelo

In [47]:
with open("data/df_test.pkl", "rb") as p:
    df_test = pickle.load(p)

In [48]:
with open("data/df_train.pkl", "rb") as p:
    df_train = pickle.load(p)

In [49]:
df_train.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,sd14,sd15,sd16,sd17,sd18,sd19,sd20,sd21,rul,failure_label
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,191,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,5.041671,0.008697,0.0,0.000000,0.0,0.0,0.042426,0.003253,190,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,3.717450,0.007640,0.0,1.154701,0.0,0.0,0.055076,0.044573,189,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,3.050906,0.028117,0.0,1.000000,0.0,0.0,0.076322,0.037977,188,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2.651326,0.025953,0.0,1.095445,0.0,0.0,0.073621,0.033498,187,0


In [50]:
# separa feature de target
features = ['engine1', 'engine2', 'engine3', 't1', 't2',
       't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13',
       't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 'av1', 'av2',
       'av3', 'av4', 'av5', 'av6', 'av7', 'av8', 'av9', 'av10', 'av11', 'av12',
       'av13', 'av14', 'av15', 'av16', 'av17', 'av18', 'av19', 'av20', 'av21',
       'sd1', 'sd2', 'sd3', 'sd4', 'sd5', 'sd6', 'sd7', 'sd8', 'sd9', 'sd10',
       'sd11', 'sd12', 'sd13', 'sd14', 'sd15', 'sd16', 'sd17', 'sd18', 'sd19',
       'sd20', 'sd21']

target = 'failure_label'

In [51]:
X = df_train[features]
y = df_train[target]

In [52]:
# separação de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.4, random_state=42)

### Regressão logistica

Resultados:
* Classe 0:
    * 98% de f1-score
    * 136 de FN
* Classe 1:
    * 83% f1-score
    * 140 FP

In [53]:
model_name= 'Logistic Regression'
model = LogisticRegression(random_state=42)
params = {'C': [.01, 0.1, 1.0, 10], 'solver': ['liblinear', 'lbfgs']}
score = 'f1'

logistic_regression = ModelFactory(model_name, model, params, score)


In [54]:
lr_model, lr_pred = logistic_regression.classify(X_train, y_train, X_test, y_test)
logistic_regression.metrics(y_test, lr_pred['y_pred'])

Matriz de confusão
[[7321  136]
 [ 140  656]]
------------------------------------------
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      7457
           1       0.83      0.82      0.83       796

    accuracy                           0.97      8253
   macro avg       0.90      0.90      0.90      8253
weighted avg       0.97      0.97      0.97      8253



/home/vinicius/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Random Forest

Resultados:
* Classe 0:
    * 99% de f1-score
    * 71 de FN
* Classe 1:
    * 89% f1-score
    * 98 FP

In [10]:
model_name= 'Random Forest'
model = RandomForestClassifier(n_estimators=50, random_state=42)
params = {'max_depth': [4, 5, 6, 7, 8, 9, 10, 15], 'criterion': ['gini', 'entropy']}
score = 'f1'

random_forest = ModelFactory(model_name, model, params, score)


In [11]:
model_random, pred = random_forest.classify(X_train, y_train, X_test, y_test)
random_forest.metrics(y_test, pred['y_pred'])

Matriz de confusão
[[7386   71]
 [  98  698]]
------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7457
           1       0.91      0.88      0.89       796

    accuracy                           0.98      8253
   macro avg       0.95      0.93      0.94      8253
weighted avg       0.98      0.98      0.98      8253



### Naive bayes gaussian

Resultados:
* Classe 0:
    * 96% de f1-score
    * 600 de FN
* Classe 1:
    * 73% f1-score
    * 2 FP

In [12]:
model_name= 'gaussian'
model = GaussianNB()
params = {}
score = 'f1'

gaussian = ModelFactory(model_name, model, params, score)


In [13]:
model_gaussian, pred = gaussian.classify(X_train, y_train, X_test, y_test)
gaussian.metrics(y_test, pred['y_pred'])

Matriz de confusão
[[6857  600]
 [   2  794]]
------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      7457
           1       0.57      1.00      0.73       796

    accuracy                           0.93      8253
   macro avg       0.78      0.96      0.84      8253
weighted avg       0.96      0.93      0.94      8253



### KNN

Resultados:
* Classe 0:
    * 98% de f1-score
    * 103 de FN
* Classe 1:
    * 83% f1-score
    * 154 FP
    

In [14]:
model_name= 'knn'
model = KNeighborsClassifier(n_jobs=-1)
params = {'n_neighbors': [9 , 10, 11, 12, 13]}
score = 'f1'

knn = ModelFactory(model_name, model, params, score)


In [15]:
model_knn, pred = knn.classify(X_train, y_train, X_test, y_test)
knn.metrics(y_test, pred['y_pred'])

Matriz de confusão
[[7354  103]
 [ 154  642]]
------------------------------------------
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7457
           1       0.86      0.81      0.83       796

    accuracy                           0.97      8253
   macro avg       0.92      0.90      0.91      8253
weighted avg       0.97      0.97      0.97      8253



## Conslusão

Dos modelos usado, dois se destacaram: <b>Random Forest</b> e <b>Naive bayes gaussian</b>

* Naive bayes: Boa acertividade (quase 100%) para prever se a máquina irá falhar(VP), porém com muitos casos (600) <br>
    de predições de troca de máquina desnecessárias(FN)

* Random forest: boa acertividade no VP, porém não tão preciso como o modelo Naive bayes,<br>
    porém não tem tantos casos de troca de maquina desnecessárias.

### - Com esse caso, seria necessário um stakeholder de negócio dizer qual traz mais valor para empresa: Ter uma melhor garantia (quase 100%) que a linha de produção não irá parar, porém ter maior custo para substituir máquinas (92%). 
### - Ou ter um garantia um pouco menor (88%) que a linha de produção não irá para, porém ter um custo menor de substituir máquinas (99%)
<br>

### <b> Para esse caso, irei escolher o Random Forest onde os erros estão mais distribuídos</b>

In [42]:
result = pd.DataFrame()

In [43]:
result['predicted'] = model_random.predict(df_test[features])

In [45]:
result.to_csv("classificao_ViniciusZambotti.csv", index = False)